In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os

In [2]:
BATCH_SIZE = 128
EPISODE = 500000
TEST_EPISODE = 1000
LEARNING_RATE =1e-5
Nom = 1e-5
Weight_Deacy =1e-5
GPU = 0
Margin = 1

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'AwA1_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'original_att'         #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")

    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个

########################add negative pairs#######################
#x_negative = np.empty_like(x)
#x_negative_label = np.empty_like(train_label)
#print(x.shape[0])

#for i in range(x.shape[0]):
#    pick=np.random.choice(np.where(train_label[i]!=train_id)[0], replace=True)
#    x_negative[i] = x[pick]
#    x_negative_label[i] = train_label[pick]
    
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
#train_negative_label = torch.from_numpy(x_negative_label).unsqueeze(1)
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
#print(all_attributes)
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
from my_net_2 import AttributeNetwork
from my_net_2 import TripletNetwork

In [5]:
# init network
print("init networks")
attribute_network = AttributeNetwork(85,1024,1024,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
triplet_network = TripletNetwork(attribute_network)  #metric learning   
triplet_network.cuda(GPU)   
#attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
#优化器adam 学习率 正则1e-5

#attribute_network_scheduler = StepLR(attribute_network_optim, step_size=200000, gamma=0.5)
#学习率每200k步 乘0.5
#triplet_network_optim = torch.optim.Adam(triplet_network.parameters(), lr=LEARNING_RATE)
#
triplet_network_optim = torch.optim.Adam(triplet_network.parameters(), lr=LEARNING_RATE, 
                                         weight_decay=Weight_Deacy)
triplet_network_scheduler = StepLR(triplet_network_optim , step_size=100000 , gamma=0.5)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def compute_accuracy(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0

    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = Variable(batch_features_ext).cuda(GPU).float()  # 32*1024

        #print(batch_features_ext)

        sample_features = attribute_network(Variable(sample_attributes).cuda(GPU).float())
        sample_features_ext = sample_features.repeat(batch_size, 1)
        #print(sample_features_ext.shape)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)


        _, predict_labels = torch.min(relations.data, 1)
        #print(predict_labels)
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    test_accuracy = total_rewards/1.0/test_size
    return  test_accuracy

In [8]:
print("training...")
last_accuracy = 0.0
for episode in range(EPISODE):
    #attribute_network.train()
    triplet_network_scheduler.step(episode)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    
    batch_features_negative = np.empty_like(batch_features.numpy())
    
    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_labels.numpy()]).squeeze(1)
    
    batch_features = Variable(batch_features).cuda(GPU).float()  # 32*2048
    #print(batch_labels)
    #print(batch_labels[1])
    for i in range(batch_features_negative.shape[0]):
        pick=np.random.choice(np.where(batch_labels.numpy()!=train_id)[0], replace=True)
        batch_features_negative[i] = batch_features[pick]    
    
    
    batch_features_negative = torch.from_numpy(batch_features_negative)
    batch_features_negative = Variable(batch_features_negative).cuda(GPU).float()  # 32*2048
    
    #batch_attributes = attribute_network(Variable(batch_attributes).cuda(GPU))
    batch_attributes = Variable(batch_attributes).cuda(GPU)
    
    batch_similar = F.pairwise_distance(attribute_network(batch_attributes),batch_features,2)
    print(batch_similar)
    
'''   
    dista, distb, embedded_x, embedded_y, embedded_z = triplet_network(batch_attributes, batch_features, batch_features_negative)
    
    criterion = nn.MarginRankingLoss(margin = Margin).cuda(GPU)
    
    target = torch.FloatTensor(dista.size()).fill_(-1).cuda(GPU)
    
    
    loss_triplet= criterion(dista, distb, target)
    loss_embedd = embedded_x.norm(2) + embedded_y.norm(2) + embedded_z.norm(2)
    loss = loss_triplet + Nom * loss_embedd

    # update
    triplet_network.zero_grad()
    loss.backward()
    triplet_network_optim.step()
    
    if (episode+1)%1000 == 0:
        print("episode:", episode+1, "loss", loss.data)
    if (episode+1)%1000 == 0:
        print("Testing...")
        #attribute_network.eval()
        zsl_accuracy = compute_accuracy(test_features, test_label, test_id, test_attributes)
        gzsl_unseen_accuracy = compute_accuracy(test_features, test_label, np.arange(50), attributes)
        gzsl_seen_accuracy = compute_accuracy(test_seen_features, test_seen_label, np.arange(50), attributes)
        H = 2 * gzsl_seen_accuracy * gzsl_unseen_accuracy / (gzsl_unseen_accuracy + gzsl_seen_accuracy)
        print('zsl:', zsl_accuracy)
        print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy , gzsl_seen_accuracy, H))
        print('_'*100)
''' 

training...
tensor([ 36.6364,  30.4560,  33.5539,  40.5153,  31.2724,  26.8363,
         36.2528,  35.6076,  38.5091,  35.7969,  26.6021,  35.9289,
         32.3336,  31.7845,  35.5963,  34.5246,  31.6110,  42.0622,
         31.8671,  41.8956,  32.8900,  33.5335,  26.1928,  38.2169,
         27.9862,  33.0397,  44.3365,  31.8331,  28.1621,  29.1286,
         32.0741,  30.3101,  42.4411,  28.2101,  24.4198,  36.4393,
         31.9400,  32.3198,  27.8782,  35.4386,  37.4327,  33.1105,
         26.9822,  26.5537,  37.5212,  33.5330,  36.5187,  32.4541,
         42.8350,  36.7736,  50.0659,  31.0899,  36.4515,  35.4640,
         35.2555,  29.0927,  28.9821,  36.6573,  24.8797,  24.9718,
         30.3746,  29.3429,  32.8157,  39.3758,  26.9921,  36.2786,
         29.4954,  30.9630,  31.0301,  35.6758,  35.5541,  25.2852,
         29.9217,  30.3345,  30.4527,  30.2915,  34.3877,  33.7905,
         27.4461,  32.8818,  35.5025,  34.0766,  36.5436,  30.3706,
         44.4422,  51.6940,  30.4366

tensor([ 40.1903,  34.8034,  29.7221,  38.4074,  31.5325,  36.4949,
         40.1623,  30.4654,  30.0773,  29.1631,  35.3344,  29.4334,
         34.8784,  28.9608,  33.3136,  37.1690,  29.4710,  40.1662,
         28.5788,  39.6464,  26.2722,  31.2306,  35.9846,  29.1017,
         33.7104,  28.6664,  36.3752,  26.7432,  45.9401,  25.6318,
         33.9110,  35.3048,  29.6382,  43.2777,  45.3975,  28.7868,
         37.1927,  31.6802,  36.3710,  29.0213,  44.2592,  28.0879,
         41.6053,  38.8306,  31.9507,  36.9038,  43.5668,  30.4875,
         37.1727,  39.3126,  36.7248,  29.6377,  38.7609,  53.0543,
         28.2128,  50.6158,  38.6536,  35.8817,  31.5964,  28.2994,
         39.9506,  38.2052,  36.0389,  34.2431,  31.2579,  29.3429,
         27.0518,  37.2577,  36.0304,  29.6503,  28.4368,  33.7263,
         32.9987,  38.6930,  36.7833,  28.2077,  30.1411,  36.2027,
         28.6043,  36.5094,  37.0876,  25.9922,  41.3085,  27.7360,
         36.5851,  30.1265,  32.1325,  50.4776, 

tensor([ 29.4191,  37.5066,  35.1554,  26.3977,  38.5316,  32.0288,
         36.4099,  32.3565,  29.7263,  39.7662,  30.0055,  35.8960,
         30.4094,  28.0177,  36.3469,  44.2975,  29.5560,  26.6955,
         35.7935,  39.2662,  36.7701,  33.6696,  32.7266,  31.2838,
         30.7603,  32.3813,  34.3990,  34.5009,  29.7953,  29.6316,
         34.2648,  44.6764,  34.2757,  27.0164,  30.4737,  27.5994,
         41.8739,  35.3444,  43.0684,  37.6542,  28.5787,  28.6335,
         26.7334,  28.5806,  26.9731,  41.0368,  32.6540,  27.3971,
         39.6831,  30.0210,  35.0028,  33.2444,  33.9146,  38.8077,
         34.4491,  46.1517,  42.4541,  26.1790,  34.4311,  31.3032,
         41.6165,  41.2612,  38.1002,  31.4073,  30.2959,  30.3625,
         27.0058,  29.7093,  39.3928,  35.5967,  35.8655,  38.3796,
         26.0505,  37.3001,  38.5665,  43.4403,  33.5907,  29.1070,
         34.5409,  38.7845,  35.0617,  28.8868,  31.8548,  36.5237,
         36.6101,  31.9004,  32.3158,  32.0035, 

tensor([ 36.8713,  31.9251,  42.3802,  37.7048,  24.4993,  30.6695,
         37.4449,  31.3283,  41.5925,  30.1167,  32.0407,  39.2530,
         42.6893,  45.9415,  27.6359,  35.8850,  26.9868,  30.2855,
         28.4350,  31.8335,  24.0628,  32.7258,  39.4322,  43.7115,
         26.4359,  40.2480,  32.7711,  35.2692,  26.0540,  28.9796,
         44.1281,  35.5417,  40.2416,  35.8471,  28.4134,  32.9351,
         32.5442,  32.8670,  28.9701,  31.3321,  41.9437,  22.8240,
         32.4541,  29.5018,  41.7344,  43.2055,  40.9226,  45.5375,
         26.0505,  35.7224,  42.8343,  31.1891,  31.6156,  42.5307,
         40.1908,  36.4790,  34.8898,  33.7108,  28.3109,  30.6891,
         29.5205,  46.1950,  42.4594,  28.2447,  37.5324,  30.1744,
         28.1999,  32.8997,  28.6716,  44.8005,  32.8309,  27.9425,
         37.7890,  23.9187,  35.0842,  36.8083,  24.2173,  30.4036,
         33.5645,  32.9565,  33.8431,  33.9585,  31.8396,  26.7611,
         26.0152,  38.5124,  37.4905,  30.4765, 

tensor([ 29.1822,  25.8349,  41.8994,  30.2151,  36.2137,  32.2637,
         38.4036,  38.5232,  30.4464,  41.5383,  35.8782,  43.3369,
         30.4304,  29.5156,  38.1769,  30.8537,  27.8611,  43.5541,
         22.4874,  28.8139,  42.0836,  32.3522,  33.1500,  36.2514,
         25.2311,  27.6569,  38.7663,  32.7735,  40.5971,  37.1208,
         47.2607,  36.9573,  31.3888,  42.7202,  41.3311,  40.8210,
         36.0570,  36.5094,  37.7033,  28.5453,  25.4393,  37.8566,
         37.8923,  32.2323,  34.1666,  31.2902,  25.5602,  36.3493,
         34.9659,  29.9460,  29.6684,  32.2161,  41.0108,  43.7724,
         31.0163,  25.2998,  33.9585,  34.7902,  31.1923,  38.3347,
         32.2213,  25.4877,  39.0499,  32.0300,  33.5515,  31.8352,
         35.4719,  38.7562,  42.3959,  36.0541,  41.6665,  31.2092,
         34.8336,  24.0306,  29.8778,  27.3609,  34.8411,  26.5393,
         32.7604,  30.4348,  30.3405,  30.9773,  33.3768,  45.2752,
         29.4000,  34.6267,  38.6648,  28.0307, 

...

tensor([ 41.3106,  46.3359,  27.7113,  29.2443,  32.0874,  33.0622,
         30.1563,  28.2025,  33.8566,  42.6867,  34.6295,  29.1068,
         28.0943,  28.5713,  30.8453,  30.1089,  30.8122,  40.7195,
         35.3999,  36.8051,  27.0766,  26.7921,  37.6403,  47.4862,
         29.5645,  34.1657,  31.1321,  39.9626,  25.0867,  25.1689,
         37.6664,  42.2364,  34.5617,  31.6802,  38.8527,  29.8833,
         40.5596,  31.3879,  30.7022,  35.7199,  36.4964,  31.0898,
         34.0655,  32.8333,  26.5581,  32.2585,  23.7644,  37.7670,
         41.7573,  34.4350,  30.4751,  35.3600,  38.6606,  25.9990,
         27.0488,  26.6307,  25.4601,  40.1515,  37.2023,  27.2676,
         34.0637,  37.9180,  37.1123,  30.4561,  25.9460,  40.4750,
         44.3315,  36.8326,  36.0498,  35.9783,  44.0603,  32.4143,
         33.2654,  30.3026,  29.7648,  29.7017,  36.4433,  28.4935,
         30.4757,  27.1225,  37.4684,  31.6553,  32.4453,  39.5209,
         33.2442,  43.4267,  28.2362,  30.1991, 

tensor([ 26.9454,  39.1202,  38.9621,  30.1213,  34.9037,  30.4810,
         32.0483,  31.3756,  32.6928,  28.7211,  28.1619,  38.6805,
         31.5749,  34.6348,  27.7954,  34.7129,  36.5758,  39.6064,
         30.5266,  29.1863,  29.9550,  27.8782,  27.4735,  38.7865,
         37.4060,  29.2327,  38.5051,  43.2368,  46.2705,  27.7919,
         31.1540,  32.1882,  49.0223,  46.5036,  37.4453,  30.7973,
         33.6673,  27.2852,  29.5095,  41.5359,  28.2488,  38.0064,
         31.8924,  27.5137,  25.6956,  38.3851,  37.8243,  30.3988,
         24.4010,  38.5193,  33.8889,  30.8355,  26.6105,  34.2088,
         31.2223,  35.1579,  35.3629,  39.1803,  34.7936,  29.7270,
         40.0657,  41.3891,  39.2319,  30.9567,  33.2672,  35.2877,
         33.3495,  30.1094,  34.4721,  37.0515,  26.5701,  36.7016,
         33.0562,  30.8819,  30.7526,  31.3565,  29.5091,  38.8717,
         35.8673,  30.0491,  39.6831,  31.6103,  27.8016,  28.4263,
         31.4908,  30.9419,  41.2904,  34.4729, 

tensor([ 28.2946,  33.4733,  39.3943,  29.3813,  40.3948,  33.7783,
         30.7860,  40.8581,  32.0383,  33.2035,  33.8634,  30.6693,
         40.1546,  34.0643,  33.3448,  24.0560,  26.9347,  26.9148,
         25.6603,  26.1356,  32.7859,  24.4008,  34.0400,  29.5898,
         34.4349,  34.1662,  31.1679,  32.0771,  28.1244,  33.3065,
         38.1627,  40.3040,  41.1208,  28.1067,  29.5280,  40.6936,
         38.2025,  27.3992,  29.8103,  31.7388,  24.8166,  39.3130,
         30.5897,  29.0058,  38.1049,  32.0450,  39.1302,  32.4307,
         31.7407,  42.6878,  29.8048,  34.8131,  29.8698,  49.9283,
         26.7283,  32.7091,  35.9274,  32.1425,  32.1926,  32.8983,
         30.9874,  35.7814,  39.8332,  33.9799,  32.5321,  31.9986,
         33.4451,  29.7355,  35.4974,  29.4860,  29.1881,  24.8762,
         31.2452,  44.0847,  27.6575,  40.3460,  32.1647,  44.8556,
         28.7651,  37.7920,  30.2515,  30.3007,  29.4968,  31.1092,
         27.3638,  29.2105,  40.8960,  40.6227, 

tensor([ 30.3019,  35.9192,  30.1126,  28.8834,  30.3988,  32.1698,
         30.9210,  35.9142,  31.9066,  33.8953,  39.6040,  39.8984,
         32.6045,  26.0249,  24.8079,  28.3211,  50.6546,  34.2122,
         31.4159,  32.0035,  36.9167,  44.0695,  42.6581,  35.2334,
         28.1646,  24.1071,  36.9282,  25.2448,  32.5306,  28.5405,
         29.3516,  41.5563,  50.0097,  44.8099,  28.2547,  36.0648,
         29.1815,  29.9747,  41.8935,  39.8824,  32.2213,  40.3625,
         37.2318,  33.9541,  34.7860,  40.1273,  34.9744,  33.9610,
         25.9615,  37.9560,  31.1122,  34.6419,  31.8024,  35.8406,
         32.7986,  27.7734,  29.0712,  28.0891,  28.0551,  30.4571,
         29.7469,  36.0073,  33.9366,  28.1348,  38.4447,  25.1819,
         30.0396,  34.2761,  39.4501,  32.0366,  24.0306,  31.3321,
         33.4819,  36.1032,  35.8952,  28.6734,  34.3110,  42.5188,
         30.7962,  39.0399,  31.0432,  30.3077,  35.1799,  27.0421,
         38.4905,  28.3580,  32.4222,  28.8284, 

KeyboardInterrupt: 